In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import cv2
import re
# import os
import glob
# import re
# import numpy as np
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
import tifffile as tiff
# import os
# import numpy as np
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
# import rasterio

import json

# with open('config.json') as json_file:
#     config = json.load(json_file)
#     corruption = config['corruption_level']


def load_image(filepath):
    # Load the image using PIL
    image = np.array(Image.open(filepath).convert('L'))
    
    # Apply Otsu's thresholding
    t, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Convert to binary (0 and 1)
    binary_image = (binary_image > 0).astype(np.uint8)
    return binary_image

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred))

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)

# Paths to the folders containing the predictions and ground truths


In [3]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)


# Directories for ground truth and prediction masks (Unet)
# gt_folder = './GEE_Masks/GEE_resized/'
# Directories for ground truth masks (DeepLabV3, Segformer, Maskformer)
gt_folder = './sar_images/masks/test'


# For UNet
# pred_folder = f'./GEE_Output/Adversarial/New_20_Epoch_{corruption}_with_diff_kernels'
# pred_files = glob.glob(os.path.join(pred_folder, 'dense_*.tif'))

# For DeepLabV3
# pred_folder = f'./GEE_Output/DeeplLabOutputs/Outputs'

# For Segformer
# pred_folder = f'./GEE_Output/Segformer/Outputs'

# For Maskformer
pred_folder = f'./GEE_Output/Maskformer/Outputs'

print(pred_folder)

# Initialize metric lists
dice_scores = []
iou_scores = []
pixel_accuracies = []
precisions = []
recalls = []
f1_scores = []
specificities = []

# Get all prediction files

# For DeepLabV3, Segformer, Maskformer
pred_files = glob.glob(os.path.join(pred_folder, '*.png'))

for pred_file in pred_files:
    # Extract the number 'i' from the prediction filename
    # For UNet
    # match = re.search(r'dense_(\d+).tif', os.path.basename(pred_file))
    # For DeeplabV3, Segformer, Maskformer
    match = re.search(r'pred_(\d+)\.png', os.path.basename(pred_file))
    
    if match:
        i = match.group(1)
        
        # For UNet
        # gt_filename = f'NDWI_Mask_{i}_resized.tif'
        
        # For Deeplabv3, Segformer and Maskformer
        gt_filename = f'{i}.png'
        
        gt_path = os.path.join(gt_folder, gt_filename)
        
        # Check if the ground truth file exists
        if os.path.exists(gt_path):
            # Load the prediction and ground truth images
            pred_image = load_image(pred_file)
            gt_image = load_image(gt_path)
            print(np.max(pred_image))
            print(np.min(pred_image))
            # Flatten the images
            pred_flat = pred_image.flatten()
            gt_flat = gt_image.flatten()
            
            # Calculate metrics
            dice = dice_coefficient(gt_flat, pred_flat)
            iou = jaccard_score(gt_flat, pred_flat, average='macro')
            accuracy = pixel_accuracy(gt_flat, pred_flat)
            precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
            recall = recall_score(gt_flat, pred_flat, average='macro')
            f1 = f1_score(gt_flat, pred_flat, average='macro')
            spec = specificity(gt_flat, pred_flat)
            
            # Store metrics
            dice_scores.append(dice)
            iou_scores.append(iou)
            pixel_accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
            specificities.append(spec)
            
            # Print metrics for the current image
            print(f"Metrics for {gt_filename} and {pred_file}:")
            print(f"  Dice Coefficient: {dice:.4f}")
            print(f"  IoU: {iou:.4f}")
            print(f"  Pixel Accuracy: {accuracy:.4f}")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall: {recall:.4f}")
            print(f"  F1 Score: {f1:.4f}")
            print(f"  Specificity: {spec:.4f}")
            print()
        else:
            print(f"Ground truth file {gt_path} not found for prediction file {pred_file}.")

# Calculate and print average metrics
average_dice = np.mean(dice_scores)
average_iou = np.mean(iou_scores)
average_accuracy = np.mean(pixel_accuracies)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)
average_specificity = np.mean(specificities)

print("Average Metrics:")
print(f"  Dice Coefficient: {average_dice:.4f}")
print(f"  IoU: {average_iou:.4f}")
print(f"  Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Precision: {average_precision:.4f}")
print(f"  Recall: {average_recall:.4f}")
print(f"  F1 Score: {average_f1:.4f}")
print(f"  Specificity: {average_specificity:.4f}")

./GEE_Output/Maskformer/Outputs
1
0
Metrics for 10.png and ./GEE_Output/Maskformer/Outputs\pred_10.png:
  Dice Coefficient: 0.7735
  IoU: 0.7110
  Pixel Accuracy: 0.8461
  Precision: 0.8694
  Recall: 0.8147
  F1 Score: 0.8285
  Specificity: 0.9640

1
0
Metrics for 1008.png and ./GEE_Output/Maskformer/Outputs\pred_1008.png:
  Dice Coefficient: 0.8800
  IoU: 0.8834
  Pixel Accuracy: 0.9822
  Precision: 0.9754
  Recall: 0.9023
  F1 Score: 0.9352
  Specificity: 0.9976

1
0
Metrics for 1010.png and ./GEE_Output/Maskformer/Outputs\pred_1010.png:
  Dice Coefficient: 0.9935
  IoU: 0.9903
  Pixel Accuracy: 0.9956
  Precision: 0.9956
  Recall: 0.9946
  F1 Score: 0.9951
  Specificity: 0.9977

1
0
Metrics for 1012.png and ./GEE_Output/Maskformer/Outputs\pred_1012.png:
  Dice Coefficient: 0.8859
  IoU: 0.8904
  Pixel Accuracy: 0.9863
  Precision: 0.9604
  Recall: 0.9203
  F1 Score: 0.9393
  Specificity: 0.9958

1
0
Metrics for 1020.png and ./GEE_Output/Maskformer/Outputs\pred_1020.png:
  Dice Coeff

c:\Users\ADMIN\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Metrics for 38.png and ./GEE_Output/Maskformer/Outputs\pred_38.png:
  Dice Coefficient: 0.9746
  IoU: 0.9452
  Pixel Accuracy: 0.9721
  Precision: 0.9715
  Recall: 0.9722
  F1 Score: 0.9718
  Specificity: 0.9730

1
0
Metrics for 385.png and ./GEE_Output/Maskformer/Outputs\pred_385.png:
  Dice Coefficient: 0.9765
  IoU: 0.9560
  Pixel Accuracy: 0.9776
  Precision: 0.9786
  Recall: 0.9771
  F1 Score: 0.9775
  Specificity: 0.9947

1
0
Metrics for 394.png and ./GEE_Output/Maskformer/Outputs\pred_394.png:
  Dice Coefficient: 0.9715
  IoU: 0.9644
  Pixel Accuracy: 0.9875
  Precision: 0.9839
  Recall: 0.9797
  F1 Score: 0.9818
  Specificity: 0.9937

1
0
Metrics for 405.png and ./GEE_Output/Maskformer/Outputs\pred_405.png:
  Dice Coefficient: 0.9662
  IoU: 0.9486
  Pixel Accuracy: 0.9756
  Precision: 0.9784
  Recall: 0.9694
  F1 Score: 0.9736
  Specificity: 0.9931

1
0
Metrics for 41.png and ./GEE_Output/Maskformer/Outputs\pred_41.png:
  Dice Coefficient: 0.7441
  IoU: 0.7523
  Pixel Accuracy:

c:\Users\ADMIN\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Metrics for 429.png and ./GEE_Output/Maskformer/Outputs\pred_429.png:
  Dice Coefficient: 1.0000
  IoU: 1.0000
  Pixel Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1 Score: 1.0000
  Specificity: 1.0000

1
0
Metrics for 43.png and ./GEE_Output/Maskformer/Outputs\pred_43.png:
  Dice Coefficient: 0.9871
  IoU: 0.9830
  Pixel Accuracy: 0.9935
  Precision: 0.9910
  Recall: 0.9918
  F1 Score: 0.9914
  Specificity: 0.9953

1
0
Metrics for 44.png and ./GEE_Output/Maskformer/Outputs\pred_44.png:
  Dice Coefficient: 0.9841
  IoU: 0.9836
  Pixel Accuracy: 0.9987
  Precision: 0.9966
  Recall: 0.9869
  F1 Score: 0.9917
  Specificity: 0.9998

1
0
Metrics for 440.png and ./GEE_Output/Maskformer/Outputs\pred_440.png:
  Dice Coefficient: 0.1779
  IoU: 0.3861
  Pixel Accuracy: 0.6856
  Precision: 0.5934
  Recall: 0.5283
  F1 Score: 0.4918
  Specificity: 0.9477

1
0
Metrics for 441.png and ./GEE_Output/Maskformer/Outputs\pred_441.png:
  Dice Coefficient: 0.9805
  IoU: 0.9791
  Pixel Accuracy:

...

Metrics for 561.png and ./GEE_Output/Maskformer/Outputs\pred_561.png:
  Dice Coefficient: 0.0106
  IoU: 0.4998
  Pixel Accuracy: 0.9996
  Precision: 0.5000
  Recall: 0.4998
  F1 Score: 0.4999
  Specificity: 0.9996

1
0
Metrics for 583.png and ./GEE_Output/Maskformer/Outputs\pred_583.png:
  Dice Coefficient: 0.9601
  IoU: 0.9506
  Pixel Accuracy: 0.9826
  Precision: 0.9825
  Recall: 0.9670
  F1 Score: 0.9745
  Specificity: 0.9951

1
0
Metrics for 584.png and ./GEE_Output/Maskformer/Outputs\pred_584.png:
  Dice Coefficient: 0.9810
  IoU: 0.9753
  Pixel Accuracy: 0.9907
  Precision: 0.9897
  Recall: 0.9852
  F1 Score: 0.9874
  Specificity: 0.9961

1
0
Metrics for 591.png and ./GEE_Output/Maskformer/Outputs\pred_591.png:
  Dice Coefficient: 0.7905
  IoU: 0.8245
  Pixel Accuracy: 0.9956
  Precision: 0.9279
  Recall: 0.8654
  F1 Score: 0.8941
  Specificity: 0.9986

1
0
Metrics for 595.png and ./GEE_Output/Maskformer/Outputs\pred_595.png:
  Dice Coefficient: 0.9595
  IoU: 0.9584
  Pixel Accur

c:\Users\ADMIN\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Metrics for 814.png and ./GEE_Output/Maskformer/Outputs\pred_814.png:
  Dice Coefficient: 0.7752
  IoU: 0.7608
  Pixel Accuracy: 0.9066
  Precision: 0.8403
  Recall: 0.8807
  F1 Score: 0.8581
  Specificity: 0.9227

1
0
Metrics for 837.png and ./GEE_Output/Maskformer/Outputs\pred_837.png:
  Dice Coefficient: 0.7373
  IoU: 0.7218
  Pixel Accuracy: 0.8829
  Precision: 0.7983
  Recall: 0.8952
  F1 Score: 0.8309
  Specificity: 0.8759

1
0
Metrics for 838.png and ./GEE_Output/Maskformer/Outputs\pred_838.png:
  Dice Coefficient: 0.3880
  IoU: 0.5529
  Pixel Accuracy: 0.8707
  Precision: 0.6229
  Recall: 0.8085
  F1 Score: 0.6578
  Specificity: 0.8785

1
0
Metrics for 846.png and ./GEE_Output/Maskformer/Outputs\pred_846.png:
  Dice Coefficient: 0.6092
  IoU: 0.6985
  Pixel Accuracy: 0.9603
  Precision: 0.9314
  Recall: 0.7283
  F1 Score: 0.7941
  Specificity: 0.9963

1
0
Metrics for 847.png and ./GEE_Output/Maskformer/Outputs\pred_847.png:
  Dice Coefficient: 0.5970
  IoU: 0.6719
  Pixel Accur

c:\Users\ADMIN\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Metrics for 957.png and ./GEE_Output/Maskformer/Outputs\pred_957.png:
  Dice Coefficient: 0.0064
  IoU: 0.4997
  Pixel Accuracy: 0.9994
  Precision: 0.4997
  Recall: 0.5000
  F1 Score: 0.4999
  Specificity: 1.0000

0
0
Metrics for 960.png and ./GEE_Output/Maskformer/Outputs\pred_960.png:
  Dice Coefficient: 0.3333
  IoU: 0.5000
  Pixel Accuracy: 1.0000
  Precision: 0.5000
  Recall: 0.5000
  F1 Score: 0.5000
  Specificity: 1.0000

1
0
Metrics for 974.png and ./GEE_Output/Maskformer/Outputs\pred_974.png:
  Dice Coefficient: 0.9886
  IoU: 0.9745
  Pixel Accuracy: 0.9873
  Precision: 0.9865
  Recall: 0.9878
  F1 Score: 0.9871
  Specificity: 0.9921

1
0
Metrics for 983.png and ./GEE_Output/Maskformer/Outputs\pred_983.png:
  Dice Coefficient: 0.9811
  IoU: 0.9622
  Pixel Accuracy: 0.9807
  Precision: 0.9807
  Recall: 0.9809
  F1 Score: 0.9807
  Specificity: 0.9880

1
0
Metrics for 986.png and ./GEE_Output/Maskformer/Outputs\pred_986.png:
  Dice Coefficient: 0.9800
  IoU: 0.9583
  Pixel Accur

In [4]:
print("Average metrics:")
print(f"  Average Dice Coefficient: {average_dice:.4f}")
print(f"  Average IoU: {average_iou:.4f}")
print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Average Precision: {average_precision:.4f}")
print(f"  Average Recall: {average_recall:.4f}")
print(f"  Average F1 Score: {average_f1:.4f}")
print(f"  Average Specificity: {average_specificity:.4f}")

# Model's overall accuracy
print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

Average metrics:
  Average Dice Coefficient: 0.7740
  Average IoU: 0.8269
  Average Pixel Accuracy: 0.9568
  Average Precision: 0.8939
  Average Recall: 0.8721
  Average F1 Score: 0.8688
  Average Specificity: 0.9727
Model's Overall Accuracy: 0.9568
